# Priority Replay

[Prioritized Experience Replay](https://arxiv.org/pdf/1511.05952.pdf) Tom Schaul, John Quan, Ioannis Antonoglou and David Silver


In [2]:
import numpy as np

## ProbabilityBag

We need to prioritize memory replay with the following requirements:
 * Memories are stored with a relative priority
 * There is an upper limit to the priorities stored in memory, and things of lower priority fall off the buttom when the memory is full
 * We can sample the memories, and the probability of any given item being selected is proportional to its priority
 * Things with lower priority still have a chance of being selected
 * Memories will get pulled out of the memory pile, then put back in based on their new updated priorities


The interface will be something like this:

In [ ]:
class ProbabilityBag:
    def __init__(self, max_size, weight_column):
        pass
    
    def pop_batch(self, n):
        """Remove a randomly selected batch of n items from the bag.  The probability
        of an item being selected is proportional to its priority.
        """
        
    def push_batch(self, items):
        """Insert a group of items.  The priority of each item is the first member of the tuple.
        Example:
            item = (priority, state, action, reward)
            items = [item]
            probability_bag.push_batch(items)
        """

## Sampling with probability
Numpy has [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html) for selecting items with a given probability:

In [3]:
a = [1,2,3,4,5,6,7,8]
weights = [3, 3, 3, 2, 2, 1, 1, 1]
sum_w = sum(weights)
p_a = [w / sum_w for w in weights]

In [4]:
sum_w, p_a

(16, [0.1875, 0.1875, 0.1875, 0.125, 0.125, 0.0625, 0.0625, 0.0625])

In [10]:
np.random.choice(a, 3, p = p_a)

array([5, 2, 5])

Notice that we got 5 twice.  This is what the *replacement* value controls; set it to false to keep the item from going back into the choice pool once selected.

In [15]:
[np.random.choice(a, 3, p=p_a, replace=False) for _ in range(5)]

[array([1, 2, 8]),
 array([2, 3, 4]),
 array([4, 2, 5]),
 array([4, 1, 6]),
 array([2, 1, 4])]

Ok, we never got duplicates.

Does ```np.random.choice``` work with plain python objects?

In [45]:
gunslingers = ['Roland','Odetta','Eddie','Jake','Oy']
weights = np.array([0.4, 0.3, 0.3, 0.3, 0.1])
print('weights',weights)
probs = weights / sum(weights)
print('probabilities',probs)
np.random.choice(gunslingers, 2, p=probs)

weights [ 0.4  0.3  0.3  0.3  0.1]
probabilities [ 0.28571429  0.21428571  0.21428571  0.21428571  0.07142857]


array(['Roland', 'Roland'],
      dtype='<U6')

Oh cool, it *does* work with plain objects.

But notice that I got the same thing twice. I need replace=False.

In [49]:
np.random.choice(gunslingers, 2, p=probs, replace=False)

array(['Eddie', 'Oy'],
      dtype='<U6')

But actually what I'm going to want to do is have it generate *array indecies* or row numbers, rather than python objects.  This is because I need to remove the corresponding items from both the weights array and the items list.

Turns out numpy supports that too, saying that if the first item is passed as an integer it behaves as if ```np.arange(n)``` were used there.

In [60]:
np.random.choice(len(gunslingers), 2, p=probs, replace=False)

array([0, 3])

## Remove an item and get its value

In [61]:
gunslingers = ['Roland','Odetta','Eddie','Jake','Oy']
index = 3
gunslingers[index]

'Jake'

In [63]:
item = gunslingers.pop(index)
item, gunslingers

('Jake', ['Roland', 'Odetta', 'Eddie', 'Oy'])

In [72]:
indexes = np.array([0,3])
indexes2 = sorted(list(indexes), reverse=True) # Theres probably a way to do this in numpy, but this is fine.
indexes2


[3, 0]

## Accessing the priority
I was considering keeping the probability as one column in a matrix.  I don't think I'll do that now, but if I do:

In [16]:
items = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [64]:
items[:,0]

TypeError: list indices must be integers or slices, not tuple

## Deleting multiple rows


In [20]:
items = np.array([(np.random.uniform(), np.random.uniform(), i) for i in range(5)])
items

array([[  1.63036584e-01,   7.11914199e-01,   0.00000000e+00],
       [  3.32402832e-01,   6.21921974e-01,   1.00000000e+00],
       [  1.75515013e-03,   8.18383945e-01,   2.00000000e+00],
       [  4.97962605e-01,   1.41039400e-01,   3.00000000e+00],
       [  1.99574623e-01,   3.28493697e-01,   4.00000000e+00]])

In [25]:
revised = np.delete(items, [0], axis=0) # Delete one item. index is zero based
revised

array([[  3.32402832e-01,   6.21921974e-01,   1.00000000e+00],
       [  1.75515013e-03,   8.18383945e-01,   2.00000000e+00],
       [  4.97962605e-01,   1.41039400e-01,   3.00000000e+00],
       [  1.99574623e-01,   3.28493697e-01,   4.00000000e+00]])

In [24]:
items # original array is not changed

array([[  1.63036584e-01,   7.11914199e-01,   0.00000000e+00],
       [  3.32402832e-01,   6.21921974e-01,   1.00000000e+00],
       [  1.75515013e-03,   8.18383945e-01,   2.00000000e+00],
       [  4.97962605e-01,   1.41039400e-01,   3.00000000e+00],
       [  1.99574623e-01,   3.28493697e-01,   4.00000000e+00]])

In [26]:
revised = np.delete(items, [1, 3], axis=0) # Delete multiple items
revised

array([[  1.63036584e-01,   7.11914199e-01,   0.00000000e+00],
       [  1.75515013e-03,   8.18383945e-01,   2.00000000e+00],
       [  1.99574623e-01,   3.28493697e-01,   4.00000000e+00]])

## Delete multiple columns

In [77]:
a = np.array([0,1,2,3,4,5])
a

array([0, 1, 2, 3, 4, 5])

In [84]:
# Determine the sum of the first n items before you delete them
np.sum(a[:4])

6

In [86]:
np.delete(a, np.s_[:4]) # Delete the first 4 items from the front

array([4, 5])

In [88]:
b=['0','1','2','3','4']
b = b[2:] # Delete first 2 items from b
b

['2', '3', '4']

## Appending to end of array

In [27]:
items = np.array([(np.random.uniform(), np.random.uniform(), i) for i in range(5)])
items

array([[ 0.77177247,  0.24157203,  0.        ],
       [ 0.36031219,  0.16887835,  1.        ],
       [ 0.96218638,  0.95175048,  2.        ],
       [ 0.48284661,  0.49979654,  3.        ],
       [ 0.61589527,  0.07276737,  4.        ]])

In [29]:
revised = np.append(items, [12, 12, 99])
revised

array([  7.71772470e-01,   2.41572029e-01,   0.00000000e+00,
         3.60312191e-01,   1.68878352e-01,   1.00000000e+00,
         9.62186376e-01,   9.51750478e-01,   2.00000000e+00,
         4.82846614e-01,   4.99796542e-01,   3.00000000e+00,
         6.15895272e-01,   7.27673690e-02,   4.00000000e+00,
         1.20000000e+01,   1.20000000e+01,   9.90000000e+01])

In [31]:
items # original array is not impacted

array([[ 0.77177247,  0.24157203,  0.        ],
       [ 0.36031219,  0.16887835,  1.        ],
       [ 0.96218638,  0.95175048,  2.        ],
       [ 0.48284661,  0.49979654,  3.        ],
       [ 0.61589527,  0.07276737,  4.        ]])

In [34]:
bag = np.array([])
bag

array([], dtype=float64)

In [37]:
np.append(bag, 7)

array([ 7.])